dropout은 학습 시 노드를 임의의 비율인 dropout rate 만큼 삭제하여 layer에 포함된 weight의 일부만 참여시킨다. 그래서 overfitting을 방지할 수 있다. 아래는 dropout rate를 바꿔줬을 때와 여기에 batch_normalization을 추가해 줬을 때의 accuracy이다.<br><br>
dropout = 0.2 accuracy = 87.54% accuracy(+batch normalization) = 88.84% <br>
dropout = 0.5 accuracy = 87.00% accuracy(+batch normalization) = 88.29% <br>
dropout = 0.8 accuracy = 84.79% accuracy(+batch normalization) = 85.20% <br><br>
dropout이 커질수록 accuracy는 점점 작아진다. dropout rate가 클수록 더 적은 데이터 unit이 학습되기 때문에 정확도는 감소한다.<br>
batch normalization은 각 batch 별로 평균과 분산을 이용하여 정규화하는 것이기 때문에 overfitting을 막을 수 있다. dropout이 노드를 일부 삭제하여 세분화시켜 앙상블 결합하는 것은 batch normalization에서 활성화 함수에 의해 최적의 분포를 만들어 특징이 잘 나타내도록 하는 것과 효과가 같다. 따라서 dropout은 dropout rate만큼 노드가 제외되기 때문에 학습 자체에서의 계산량이 줄어들지만 batch normalization은 삭제하는 노드가 없고 연산 중간마다 연산 결과를 정규화하고 각 성분의 분포 차로 인해 발생하는 가중치 학습의 불균형을 방지하기 때문에 dropout에 비해 accuracy가 높게 나온다.

In [9]:
import os, random
import keras
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.utils import np_utils
import numpy as np
import tensorflow as tf
os.path.expanduser = lambda path: './'

In [10]:
batch_size = 128
num_classes = 10
epochs = 60

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


# Define Model 0.2

In [23]:
# for reproducibility
import random, os
os.environ['PYTHONHASHSEED']='0'
random.seed(123)
np.random.seed(123)
tf.random.set_seed(123)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,    inter_op_parallelism_threads=1,
                                                               allow_soft_placement=True, device_count = {'CPU': 1}))
from tensorflow.python.keras import backend as K
K.set_session(sess)



kernel_initializer='glorot_uniform'
activation_function = 'relu'

with tf.device('/cpu:0'):
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.2))
    #model.add(BatchNormalization())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    #model.add(BatchNormalization())
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 512)               401920    
                                                                 
 dropout_12 (Dropout)        (None, 512)               0         
                                                                 
 dense_19 (Dense)            (None, 512)               262656    
                                                                 
 dropout_13 (Dropout)        (None, 512)               0         
                                                                 
 dense_20 (Dense)            (None, 10)                5130      
                                                                 
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


# Start Training

In [24]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2)

Epoch 1/60
375/375 [==============================] - 12s 32ms/step - loss: 1.1965 - accuracy: 0.6194 - val_loss: 0.7336 - val_accuracy: 0.7559
Epoch 2/60
375/375 [==============================] - 12s 31ms/step - loss: 0.7406 - accuracy: 0.7520 - val_loss: 0.6155 - val_accuracy: 0.7908
Epoch 3/60
375/375 [==============================] - 12s 32ms/step - loss: 0.6476 - accuracy: 0.7791 - val_loss: 0.5577 - val_accuracy: 0.8116
Epoch 4/60
375/375 [==============================] - 12s 32ms/step - loss: 0.5926 - accuracy: 0.7969 - val_loss: 0.5254 - val_accuracy: 0.8217
Epoch 5/60
375/375 [==============================] - 12s 32ms/step - loss: 0.5588 - accuracy: 0.8095 - val_loss: 0.5020 - val_accuracy: 0.8296
Epoch 6/60
375/375 [==============================] - 12s 32ms/step - loss: 0.5347 - accuracy: 0.8159 - val_loss: 0.4840 - val_accuracy: 0.8355
Epoch 7/60
375/375 [==============================] - 12s 32ms/step - loss: 0.5134 - accuracy: 0.8234 - val_loss: 0.4705 - val_accuracy:

375/375 [==============================] - 11s 31ms/step - loss: 0.3065 - accuracy: 0.8912 - val_loss: 0.3242 - val_accuracy: 0.8851
Epoch 58/60
375/375 [==============================] - 12s 31ms/step - loss: 0.3031 - accuracy: 0.8910 - val_loss: 0.3236 - val_accuracy: 0.8852
Epoch 59/60
375/375 [==============================] - 12s 31ms/step - loss: 0.3019 - accuracy: 0.8920 - val_loss: 0.3236 - val_accuracy: 0.8852
Epoch 60/60
375/375 [==============================] - 12s 31ms/step - loss: 0.3001 - accuracy: 0.8928 - val_loss: 0.3225 - val_accuracy: 0.8855


# Calculate accuracy

In [25]:
metrics = model.evaluate(x_test, y_test) #returns loss and accuracy
print(metrics[1])
print(f'Accuracy: {metrics[1]*100:.2f}%\n')

313/313 [==============================] - 2s 5ms/step - loss: 0.3466 - accuracy: 0.8754
0.8754000067710876
Accuracy: 87.54%



--
# Define Model 0.2 with Batch_Normalization

In [26]:
# for reproducibility
import random, os
os.environ['PYTHONHASHSEED']='0'
random.seed(123)
np.random.seed(123)
tf.random.set_seed(123)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,    inter_op_parallelism_threads=1,
                                                               allow_soft_placement=True, device_count = {'CPU': 1}))
from tensorflow.python.keras import backend as K
K.set_session(sess)



kernel_initializer='glorot_uniform'
activation_function = 'relu'

with tf.device('/cpu:0'):
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 512)               401920    
                                                                 
 dropout_14 (Dropout)        (None, 512)               0         
                                                                 
 batch_normalization_8 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dense_22 (Dense)            (None, 512)               262656    
                                                                 
 dropout_15 (Dropout)        (None, 512)               0         
                                                                 
 batch_normalization_9 (Batc  (None, 512)              2048      
 hNormalization)                                      

# Start Training

In [27]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2)

Epoch 1/60
375/375 [==============================] - 14s 34ms/step - loss: 0.7247 - accuracy: 0.7490 - val_loss: 0.5124 - val_accuracy: 0.8213
Epoch 2/60
375/375 [==============================] - 13s 35ms/step - loss: 0.5075 - accuracy: 0.8196 - val_loss: 0.4157 - val_accuracy: 0.8528
Epoch 3/60
375/375 [==============================] - 13s 36ms/step - loss: 0.4556 - accuracy: 0.8376 - val_loss: 0.3867 - val_accuracy: 0.8639
Epoch 4/60
375/375 [==============================] - 14s 37ms/step - loss: 0.4217 - accuracy: 0.8502 - val_loss: 0.3705 - val_accuracy: 0.8672
Epoch 5/60
375/375 [==============================] - 13s 36ms/step - loss: 0.4018 - accuracy: 0.8552 - val_loss: 0.3563 - val_accuracy: 0.8722
Epoch 6/60
375/375 [==============================] - 12s 33ms/step - loss: 0.3843 - accuracy: 0.8636 - val_loss: 0.3516 - val_accuracy: 0.8724
Epoch 7/60
375/375 [==============================] - 12s 33ms/step - loss: 0.3694 - accuracy: 0.8666 - val_loss: 0.3452 - val_accuracy:

375/375 [==============================] - 13s 35ms/step - loss: 0.1591 - accuracy: 0.9414 - val_loss: 0.3122 - val_accuracy: 0.8943
Epoch 58/60
375/375 [==============================] - 13s 34ms/step - loss: 0.1593 - accuracy: 0.9414 - val_loss: 0.3194 - val_accuracy: 0.8929
Epoch 59/60
375/375 [==============================] - 12s 33ms/step - loss: 0.1537 - accuracy: 0.9433 - val_loss: 0.3200 - val_accuracy: 0.8938
Epoch 60/60
375/375 [==============================] - 12s 33ms/step - loss: 0.1583 - accuracy: 0.9420 - val_loss: 0.3297 - val_accuracy: 0.8908


# Calculate accuracy 

In [28]:
metrics = model.evaluate(x_test, y_test) #returns loss and accuracy
print(metrics[1])
print(f'Accuracy: {metrics[1]*100:.2f}%\n')

313/313 [==============================] - 2s 5ms/step - loss: 0.3543 - accuracy: 0.8884
0.8884000182151794
Accuracy: 88.84%



--
# Define Model 0.5

In [29]:
# for reproducibility
import random, os
os.environ['PYTHONHASHSEED']='0'
random.seed(123)
np.random.seed(123)
tf.random.set_seed(123)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,    inter_op_parallelism_threads=1,
                                                               allow_soft_placement=True, device_count = {'CPU': 1}))
from tensorflow.python.keras import backend as K
K.set_session(sess)



kernel_initializer='glorot_uniform'
activation_function = 'relu'

with tf.device('/cpu:0'):
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.5))
    #model.add(BatchNormalization())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    #model.add(BatchNormalization())
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 512)               401920    
                                                                 
 dropout_16 (Dropout)        (None, 512)               0         
                                                                 
 dense_25 (Dense)            (None, 512)               262656    
                                                                 
 dropout_17 (Dropout)        (None, 512)               0         
                                                                 
 dense_26 (Dense)            (None, 10)                5130      
                                                                 
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


# Start Training

In [30]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2)

Epoch 1/60
375/375 [==============================] - 13s 34ms/step - loss: 1.4219 - accuracy: 0.5107 - val_loss: 0.8141 - val_accuracy: 0.7207
Epoch 2/60
375/375 [==============================] - 13s 33ms/step - loss: 0.9018 - accuracy: 0.6829 - val_loss: 0.6747 - val_accuracy: 0.7587
Epoch 3/60
375/375 [==============================] - 13s 34ms/step - loss: 0.7783 - accuracy: 0.7286 - val_loss: 0.6099 - val_accuracy: 0.7834
Epoch 4/60
375/375 [==============================] - 13s 33ms/step - loss: 0.7092 - accuracy: 0.7525 - val_loss: 0.5697 - val_accuracy: 0.7994
Epoch 5/60
375/375 [==============================] - 13s 33ms/step - loss: 0.6586 - accuracy: 0.7705 - val_loss: 0.5365 - val_accuracy: 0.8119
Epoch 6/60
375/375 [==============================] - 12s 32ms/step - loss: 0.6229 - accuracy: 0.7840 - val_loss: 0.5159 - val_accuracy: 0.8211
Epoch 7/60
375/375 [==============================] - 12s 33ms/step - loss: 0.5972 - accuracy: 0.7930 - val_loss: 0.4992 - val_accuracy:

375/375 [==============================] - 12s 31ms/step - loss: 0.3591 - accuracy: 0.8707 - val_loss: 0.3379 - val_accuracy: 0.8777
Epoch 58/60
375/375 [==============================] - 12s 32ms/step - loss: 0.3547 - accuracy: 0.8730 - val_loss: 0.3368 - val_accuracy: 0.8796
Epoch 59/60
375/375 [==============================] - 12s 32ms/step - loss: 0.3510 - accuracy: 0.8738 - val_loss: 0.3359 - val_accuracy: 0.8793
Epoch 60/60
375/375 [==============================] - 12s 32ms/step - loss: 0.3515 - accuracy: 0.8744 - val_loss: 0.3347 - val_accuracy: 0.8798


# Calculate accuracy 

In [31]:
metrics = model.evaluate(x_test, y_test) #returns loss and accuracy
print(metrics[1])
print(f'Accuracy: {metrics[1]*100:.2f}%\n')

313/313 [==============================] - 2s 5ms/step - loss: 0.3616 - accuracy: 0.8700
0.8700000047683716
Accuracy: 87.00%



--
# Define Model 0.5 with Batch_Nomalization

In [32]:
# for reproducibility
import random, os
os.environ['PYTHONHASHSEED']='0'
random.seed(123)
np.random.seed(123)
tf.random.set_seed(123)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,    inter_op_parallelism_threads=1,
                                                               allow_soft_placement=True, device_count = {'CPU': 1}))
from tensorflow.python.keras import backend as K
K.set_session(sess)



kernel_initializer='glorot_uniform'
activation_function = 'relu'

with tf.device('/cpu:0'):
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 512)               401920    
                                                                 
 dropout_18 (Dropout)        (None, 512)               0         
                                                                 
 batch_normalization_10 (Bat  (None, 512)              2048      
 chNormalization)                                                
                                                                 
 dense_28 (Dense)            (None, 512)               262656    
                                                                 
 dropout_19 (Dropout)        (None, 512)               0         
                                                                 
 batch_normalization_11 (Bat  (None, 512)              2048      
 chNormalization)                                     

# Start Training

In [33]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2)

Epoch 1/60
375/375 [==============================] - 13s 34ms/step - loss: 1.0646 - accuracy: 0.6363 - val_loss: 0.6026 - val_accuracy: 0.7899
Epoch 2/60
375/375 [==============================] - 12s 33ms/step - loss: 0.6901 - accuracy: 0.7556 - val_loss: 0.4848 - val_accuracy: 0.8268
Epoch 3/60
375/375 [==============================] - 12s 33ms/step - loss: 0.6121 - accuracy: 0.7825 - val_loss: 0.4491 - val_accuracy: 0.8388
Epoch 4/60
375/375 [==============================] - 12s 33ms/step - loss: 0.5659 - accuracy: 0.7996 - val_loss: 0.4331 - val_accuracy: 0.8446
Epoch 5/60
375/375 [==============================] - 12s 33ms/step - loss: 0.5367 - accuracy: 0.8098 - val_loss: 0.4151 - val_accuracy: 0.8519
Epoch 6/60
375/375 [==============================] - 13s 34ms/step - loss: 0.5126 - accuracy: 0.8180 - val_loss: 0.4078 - val_accuracy: 0.8537
Epoch 7/60
375/375 [==============================] - 13s 36ms/step - loss: 0.4909 - accuracy: 0.8259 - val_loss: 0.4011 - val_accuracy:

375/375 [==============================] - 12s 33ms/step - loss: 0.2981 - accuracy: 0.8910 - val_loss: 0.3090 - val_accuracy: 0.8878
Epoch 58/60
375/375 [==============================] - 13s 34ms/step - loss: 0.2940 - accuracy: 0.8915 - val_loss: 0.3091 - val_accuracy: 0.8869
Epoch 59/60
375/375 [==============================] - 13s 34ms/step - loss: 0.2930 - accuracy: 0.8908 - val_loss: 0.3078 - val_accuracy: 0.8896
Epoch 60/60
375/375 [==============================] - 12s 33ms/step - loss: 0.2944 - accuracy: 0.8907 - val_loss: 0.3093 - val_accuracy: 0.8894


# Calculate accuracy

In [34]:
metrics = model.evaluate(x_test, y_test) #returns loss and accuracy
print(metrics[1])
print(f'Accuracy: {metrics[1]*100:.2f}%\n')

313/313 [==============================] - 2s 5ms/step - loss: 0.3351 - accuracy: 0.8829
0.8828999996185303
Accuracy: 88.29%



--
# Define Model 0.8

In [35]:
# for reproducibility
import random, os
os.environ['PYTHONHASHSEED']='0'
random.seed(123)
np.random.seed(123)
tf.random.set_seed(123)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,    inter_op_parallelism_threads=1,
                                                               allow_soft_placement=True, device_count = {'CPU': 1}))
from tensorflow.python.keras import backend as K
K.set_session(sess)



kernel_initializer='glorot_uniform'
activation_function = 'relu'

with tf.device('/cpu:0'):
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.8))
    #model.add(BatchNormalization())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.8))
    #model.add(BatchNormalization())
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 512)               401920    
                                                                 
 dropout_20 (Dropout)        (None, 512)               0         
                                                                 
 dense_31 (Dense)            (None, 512)               262656    
                                                                 
 dropout_21 (Dropout)        (None, 512)               0         
                                                                 
 dense_32 (Dense)            (None, 10)                5130      
                                                                 
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


# Start Training

In [36]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2)

Epoch 1/60
375/375 [==============================] - 12s 32ms/step - loss: 1.9857 - accuracy: 0.2964 - val_loss: 1.2485 - val_accuracy: 0.6533
Epoch 2/60
375/375 [==============================] - 12s 31ms/step - loss: 1.3894 - accuracy: 0.4931 - val_loss: 0.9019 - val_accuracy: 0.6883
Epoch 3/60
375/375 [==============================] - 12s 31ms/step - loss: 1.1606 - accuracy: 0.5704 - val_loss: 0.7841 - val_accuracy: 0.7173
Epoch 4/60
375/375 [==============================] - 12s 31ms/step - loss: 1.0342 - accuracy: 0.6184 - val_loss: 0.7244 - val_accuracy: 0.7366
Epoch 5/60
375/375 [==============================] - 12s 31ms/step - loss: 0.9530 - accuracy: 0.6458 - val_loss: 0.6814 - val_accuracy: 0.7552
Epoch 6/60
375/375 [==============================] - 12s 32ms/step - loss: 0.8954 - accuracy: 0.6679 - val_loss: 0.6516 - val_accuracy: 0.7627
Epoch 7/60
375/375 [==============================] - 12s 31ms/step - loss: 0.8532 - accuracy: 0.6861 - val_loss: 0.6264 - val_accuracy:

375/375 [==============================] - 12s 31ms/step - loss: 0.4954 - accuracy: 0.8282 - val_loss: 0.4004 - val_accuracy: 0.8575
Epoch 58/60
375/375 [==============================] - 12s 31ms/step - loss: 0.4924 - accuracy: 0.8288 - val_loss: 0.3997 - val_accuracy: 0.8589
Epoch 59/60
375/375 [==============================] - 12s 31ms/step - loss: 0.4882 - accuracy: 0.8296 - val_loss: 0.3995 - val_accuracy: 0.8591
Epoch 60/60
375/375 [==============================] - 12s 31ms/step - loss: 0.4878 - accuracy: 0.8294 - val_loss: 0.3971 - val_accuracy: 0.8596


# Calculate accuracy

In [37]:
metrics = model.evaluate(x_test, y_test) #returns loss and accuracy
print(metrics[1])
print(f'Accuracy: {metrics[1]*100:.2f}%\n')

313/313 [==============================] - 2s 5ms/step - loss: 0.4214 - accuracy: 0.8479
0.8478999733924866
Accuracy: 84.79%



--
# Define Model 0.8 with Batch_Nomalization

In [38]:
# for reproducibility
import random, os
os.environ['PYTHONHASHSEED']='0'
random.seed(123)
np.random.seed(123)
tf.random.set_seed(123)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,    inter_op_parallelism_threads=1,
                                                               allow_soft_placement=True, device_count = {'CPU': 1}))
from tensorflow.python.keras import backend as K
K.set_session(sess)



kernel_initializer='glorot_uniform'
activation_function = 'relu'

with tf.device('/cpu:0'):
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.8))
    model.add(BatchNormalization())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.8))
    model.add(BatchNormalization())
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_33 (Dense)            (None, 512)               401920    
                                                                 
 dropout_22 (Dropout)        (None, 512)               0         
                                                                 
 batch_normalization_12 (Bat  (None, 512)              2048      
 chNormalization)                                                
                                                                 
 dense_34 (Dense)            (None, 512)               262656    
                                                                 
 dropout_23 (Dropout)        (None, 512)               0         
                                                                 
 batch_normalization_13 (Bat  (None, 512)              2048      
 chNormalization)                                    

# Start Training

In [39]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2)

Epoch 1/60
375/375 [==============================] - 13s 33ms/step - loss: 2.0183 - accuracy: 0.3194 - val_loss: 1.0268 - val_accuracy: 0.6763
Epoch 2/60
375/375 [==============================] - 12s 33ms/step - loss: 1.3020 - accuracy: 0.5308 - val_loss: 0.7797 - val_accuracy: 0.7347
Epoch 3/60
375/375 [==============================] - 12s 33ms/step - loss: 1.0812 - accuracy: 0.6007 - val_loss: 0.7038 - val_accuracy: 0.7508
Epoch 4/60
375/375 [==============================] - 12s 33ms/step - loss: 0.9761 - accuracy: 0.6415 - val_loss: 0.6669 - val_accuracy: 0.7650
Epoch 5/60
375/375 [==============================] - 12s 33ms/step - loss: 0.8963 - accuracy: 0.6697 - val_loss: 0.6340 - val_accuracy: 0.7774
Epoch 6/60
375/375 [==============================] - 12s 33ms/step - loss: 0.8495 - accuracy: 0.6876 - val_loss: 0.6127 - val_accuracy: 0.7822
Epoch 7/60
375/375 [==============================] - 12s 33ms/step - loss: 0.8104 - accuracy: 0.7042 - val_loss: 0.5925 - val_accuracy:

375/375 [==============================] - 12s 32ms/step - loss: 0.5050 - accuracy: 0.8239 - val_loss: 0.4053 - val_accuracy: 0.8566
Epoch 58/60
375/375 [==============================] - 12s 33ms/step - loss: 0.5008 - accuracy: 0.8253 - val_loss: 0.4041 - val_accuracy: 0.8565
Epoch 59/60
375/375 [==============================] - 12s 33ms/step - loss: 0.4948 - accuracy: 0.8275 - val_loss: 0.4045 - val_accuracy: 0.8567
Epoch 60/60
375/375 [==============================] - 12s 33ms/step - loss: 0.4991 - accuracy: 0.8240 - val_loss: 0.3992 - val_accuracy: 0.8593


# Calculate accuracy

In [40]:
metrics = model.evaluate(x_test, y_test) #returns loss and accuracy
print(metrics[1])
print(f'Accuracy: {metrics[1]*100:.2f}%\n')

313/313 [==============================] - 2s 5ms/step - loss: 0.4211 - accuracy: 0.8520
0.8519999980926514
Accuracy: 85.20%

